# 1 Processing

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

In [ ]:
#clearml mac
# %env CLEARML_WEB_HOST=https://app.clear.ml
# %env CLEARML_API_HOST=https://api.clear.ml
# %env CLEARML_FILES_HOST=https://files.clear.ml
# %env CLEARML_API_ACCESS_KEY=QO1VBBX9J2S2VYILQTGI
# %env CLEARML_API_SECRET_KEY=ERuc1S6o5SirQGugvYXDFjH9b9aNi0u8S3rpALzXMa8YPSLDMW

# clearml win
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=Z4YDBU13VPOFHBHF8667
%env CLEARML_API_SECRET_KEY=JMzvRXn76AT83WuFJfS0FBGCY8c5TccbH5XboTYztrWqwzDdyn

In [ ]:
# Pip install method (recommended)

!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
# read config file
import yaml

def load_config(config_file):
    with open(config_file, 'r') as f:
        config = yaml.safe_load(f)
    return config

config = load_config('SF_config_7.yaml')

print(config)

# project
project = config['project']
name = config['name']
exist_ok = config['exist_ok']

# dataset
# data = config['data_mac']
data = config['data_win']
test_data = config['test_data']
imgsz = config['imgsz']

# model
model = config['model']
task = config['task']
mode = config['mode']
# device = config['device_mac']
device = config['device_win']

# Training parameters
epochs = config['epochs']
batch = config['batch']
lr0 = config['lr0']
cos_lr = config['cos_lr']
weight_decay = config['weight_decay']
dropout = config['dropout']
optimizer = config['optimizer']
momentum = config['momentum']
seed = config['seed']

# Output configuration
# save = config['save']
# save_period = config['save_period']
plots = config['plots']
test_output = config['test_output']


In [ ]:
from clearml import Task

#Clear ML Initialization
cl_task = Task.init(project_name=project,task_name=name)
logger = cl_task.get_logger()

# 2 Training

In [ ]:
from ultralytics import YOLO

model = YOLO(model)

results = model.train(project=project, name=name, exist_ok=exist_ok, data=data, imgsz=imgsz, task=task, device=device, epochs=epochs, batch=batch, lr0=lr0, cos_lr=cos_lr, weight_decay=weight_decay, dropout=dropout, optimizer=optimizer, momentum=momentum, seed=seed, plots=plots)

cl_task.close()

# 3 testing

## 3.1 path

In [7]:
from ultralytics import YOLO
import cv2
import os

model_path = "/Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Detection/YOLO/202403_Schweinfurt/Uni-Wood/Schweinfurt-Yolo-Aug-Test-5/weights/best.pt"

image_folder = "/Users/holmes/Desktop/probe"

test_data= "/Users/holmes/Desktop/probe/warped_image.png"

save_path = "/Users/holmes/Desktop/probe"



## 3.2 resize

In [8]:
# Resize images
resized_image_folder = os.path.join(image_folder, "resized")
os.makedirs(resized_image_folder, exist_ok=True)

for file in os.listdir(image_folder):
    if file.endswith(('.jpg', '.JPG', '.jpeg', '.png', '.bmp')):
        img_path = os.path.join(image_folder, file)
        img = cv2.imread(img_path)
        img_resized = cv2.resize(img, (640, 640))
        resized_img_path = os.path.join(resized_image_folder, os.path.splitext(file)[0] + "_resize" + os.path.splitext(file)[1])
        cv2.imwrite(resized_img_path, img_resized)
image_folder = resized_image_folder


## 3.3 test

In [2]:
model = YOLO(model_path)

if not os.path.exists(save_path):
    os.makedirs(save_path)

image_files = [os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith(('.jpg', '.jpeg', '.JPG', '.png', '.bmp'))]

# 处理每张图片
for img_path in image_files:
    results = model(img_path)
    img = cv2.imread(img_path)

    for result in results:
        boxes = result.boxes.cpu().numpy()
        for box in boxes:
            # 获取预测信息
            confidence = box.conf[0]
            if confidence > 0.4:
                xyxy = box.xyxy[0].astype(int)
                confidence_text = f"{confidence:.2f}"

                # 画出边界框和置信度
                cv2.rectangle(img, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 0, 255), 2)
                cv2.putText(img, confidence_text, (xyxy[0], xyxy[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

    # 生成唯一的文件名
    unique_save_path = os.path.join(save_path, f"1_{os.path.basename(img_path)}")
    cv2.imwrite(unique_save_path, img)
    print(f"Image saved: {unique_save_path}")


image 1/1 /Users/holmes/Desktop/probe/warped_image.png: 640x640 (no detections), 394.4ms
Speed: 3.6ms preprocess, 394.4ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)
Image saved: /Users/holmes/Desktop/probe/1_warped_image.png


In [14]:
model = YOLO(model_path)

results = model(test_data, save_dir=save_path)

for r in results:

    for box in r.boxes:

        coordinates = (box.xyxy).tolist()[0]

        print(coordinates)

        left, top, right, bottom = coordinates[0], coordinates[1], coordinates[2], coordinates[3]

        cv2.rectangle(img, (int(left), int(top)), (int(right), int(bottom)), (255, 0, 0), 2)

        cv2.imshow('window', img)



image 1/1 /Users/holmes/Desktop/probe/warped_image.png: 640x640 1 wood_knot, 400.6ms
Speed: 3.1ms preprocess, 400.6ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
[345.9314880371094, 288.6927795410156, 409.6512756347656, 346.50323486328125]


In [1]:
from ultralytics import YOLO
import cv2
import os

model_path = "/Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Detection/YOLO/202403_Schweinfurt/Uni-Wood/Schweinfurt-Yolo-Aug-Test-5/weights/best.pt"

image_folder = "/Users/holmes/Desktop/probe"
test_data = "/Users/holmes/Desktop/probe/warped_image.png"

save_path = "/Users/holmes/Desktop/probe"
if not os.path.exists(save_path):
    os.makedirs(save_path)

# 加载模型
model = YOLO(model_path)

results = model(test_data)

img = cv2.imread(test_data)

for r in results:
    for box in r.boxes:
        coordinates = (box.xyxy).tolist()[0]
        left, top, right, bottom = coordinates[0], coordinates[1], coordinates[2], coordinates[3]

        cv2.rectangle(img, (int(left), int(top)), (int(right), int(bottom)), (0, 255, 0), 2)

        confidence = box.conf[0]
        confidence_text = f"{confidence:.2f}"
        cv2.putText(img, confidence_text, (int(left), int(top) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

output_filename = os.path.join(save_path, "warped_image_with_boxes.png")
cv2.imwrite(output_filename, img)
print(f"Image saved: {output_filename}")

# cv2.imshow('Detected Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


image 1/1 /Users/holmes/Desktop/probe/warped_image.png: 640x640 1 wood_knot, 403.9ms
Speed: 3.0ms preprocess, 403.9ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)
Image saved: /Users/holmes/Desktop/probe/warped_image_with_boxes.png
